In [237]:
import pandas as pd
import json
import numpy as np
import folium
import nltk
import time
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import math
from geopy.geocoders import Nominatim
import re
from mpl_toolkits.basemap import Basemap


In [238]:
#df = pd.read_csv('tweets_en_april.csv',  header= -1)

df = pd.read_csv('processed/sentiments/twitter_sentiments_combined.csv')
#df.columns = ["City", "Sentiment", "Count"]
df.head()



,City,Sentiment,Count,Month,Language
0,Luzern,NEUTRAL,3069,april,de
1,Switzerland,POSITIVE,2246,april,de
2,Uster,POSITIVE,8,april,de
3,Winterthur,POSITIVE,83,april,de
4,Sursee,POSITIVE,3,april,de


In [239]:
switzerland = ['Schweiz', 'Schwyz', 'سويسرا', 'สวิตเซอร์แลนด์', 'スイス','Thuy Si','CH',
       'Suica', 'Suisse', 'Suiza', 'Suíça', 'Suïssa', 'Svajcarska',
       'Sveitsi', 'Svizzera', 'Swiss', 'Switzerland',
       'Zwitserland', 'Zürich', 'İsviçre', 'Швейцария']

In [240]:
df['City'] = df['City'].map(lambda x: re.sub(r'[^.áâçàéèa-zA-ZÖöÜüÄä]+', ' ', x))
#df['City'] = df['City'].map(lambda x: x.split(None, 1)[0])
df['City'] = df['City'].map(lambda x: x.split(' ', 1)[0])
df['City'] = df['City'].map(lambda x: x.lower())
df = df.loc[~df['City'].isin(switzerland)]
#df['City'].replace(' ', np.nan, inplace=False)
df = df[pd.notnull(df['City'])]

df

,City,Sentiment,Count,Month,Language
0,luzern,NEUTRAL,3069,april,de
1,switzerland,POSITIVE,2246,april,de
2,uster,POSITIVE,8,april,de
3,winterthur,POSITIVE,83,april,de
4,sursee,POSITIVE,3,april,de
5,saanen,NEUTRAL,74,april,de
6,baden,NEUTRAL,1398,april,de
7,meilen,NEUTRAL,8,april,de
8,svajcarska,NEUTRAL,4,april,de
9,waldenburg,NEUTRAL,659,april,de


In [241]:
trial = df.ix[:5]
trial


,City,Sentiment,Count,Month,Language
0,luzern,NEUTRAL,3069,april,de
1,switzerland,POSITIVE,2246,april,de
2,uster,POSITIVE,8,april,de
3,winterthur,POSITIVE,83,april,de
4,sursee,POSITIVE,3,april,de
5,saanen,NEUTRAL,74,april,de


In [242]:
df = df.loc[df['Sentiment'].isin(['POSITIVE','NEGATIVE'])]
df.head()





,City,Sentiment,Count,Month,Language
1,switzerland,POSITIVE,2246,april,de
2,uster,POSITIVE,8,april,de
3,winterthur,POSITIVE,83,april,de
4,sursee,POSITIVE,3,april,de
15,biel,POSITIVE,5,april,de


In [246]:
df['City'] = pd.core.strings.str_strip(df['City'])
df.head()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,City,Sentiment,Count,Month,Language
1,switzerland,POSITIVE,2246,april,de
2,uster,POSITIVE,8,april,de
3,winterthur,POSITIVE,83,april,de
4,sursee,POSITIVE,3,april,de
15,biel,POSITIVE,5,april,de


In [247]:
df = df.groupby(['City','Sentiment'])[('Count')].sum()
df

City              Sentiment
                  NEGATIVE      3020
                  POSITIVE      7065
.                 NEGATIVE        13
                  POSITIVE        21
..                POSITIVE         1
...               NEGATIVE         3
                  POSITIVE         2
...anti           POSITIVE         1
.ch               NEGATIVE         3
                  POSITIVE        14
.fr               NEGATIVE        20
                  POSITIVE        78
a                 NEGATIVE         9
                  POSITIVE        36
aachen            POSITIVE         3
aadorf            NEGATIVE         2
                  POSITIVE        47
aarau             NEGATIVE       389
                  POSITIVE      1164
aarberg           NEGATIVE         1
                  POSITIVE         4
aarburg           NEGATIVE        53
                  POSITIVE        84
aargau            NEGATIVE       978
                  POSITIVE      1231
aarhus            POSITIVE         1
abadons   

In [248]:
df = df.reset_index()
df

,City,Sentiment,Count
0,,NEGATIVE,3020
1,,POSITIVE,7065
2,.,NEGATIVE,13
3,.,POSITIVE,21
4,..,POSITIVE,1
5,...,NEGATIVE,3
6,...,POSITIVE,2
7,...anti,POSITIVE,1
8,.ch,NEGATIVE,3
9,.ch,POSITIVE,14


In [251]:
a = df.loc[df.groupby("City")["Count"].idxmax()]
a.head(20)


,City,Sentiment,Count
1,,POSITIVE,7065
3,.,POSITIVE,21
4,..,POSITIVE,1
5,...,NEGATIVE,3
7,...anti,POSITIVE,1
9,.ch,POSITIVE,14
11,.fr,POSITIVE,78
13,a,POSITIVE,36
14,aachen,POSITIVE,3
16,aadorf,POSITIVE,47


In [254]:
geolocator = Nominatim()

def getCoordinates (row):
    #time.sleep(0.01)
    #print(row['City'])
    try:
        address, (x, y) = geolocator.geocode(row['City'])
        address = address.split(",")
        country = address[-3]
        return x, y, country
    except Exception:
        return np.nan, np.nan, " "
     


### The above algorithm runs very long for a large dataframe!
The solution is in file: coordinatesAndCountriesGivenSentiments10m.csv

In [257]:
#a["Latitude"], a["Longitude"], a["Country"]= zip(*a.apply (lambda row: getCoordinates (row),axis=1))
#a

In [284]:
a = pd.read_csv('coordinatesAndCountriesGivenSentiments10m.csv')
a['Country'].dtype
a.head(20)

,City,Sentiment,Count,Latitude,Longitude,Country
0,NaN,POSITIVE,7065,NaN,NaN,
1,.,POSITIVE,21,NaN,NaN,
2,..,POSITIVE,1,NaN,NaN,
3,...,NEGATIVE,3,NaN,NaN,
4,...Anti,POSITIVE,1,31.105040,77.773590,Shimla
5,.ch,POSITIVE,14,46.798562,8.231974,Suisse
6,.fr,POSITIVE,78,NaN,NaN,
7,A,POSITIVE,25,NaN,NaN,
8,ABOUT,POSITIVE,1,2.908066,11.214553,Mvila
9,AC,NEGATIVE,7,NaN,NaN,


### Find only the data for Switzerland

In [289]:
a['Country'] = pd.core.strings.str_strip(a['Country'])
a['City'] = pd.core.strings.str_strip(a['City'])
allSuisse = a.loc[a['Country'] == 'Suisse']
allSuisse.head()

,City,Sentiment,Count,Latitude,Longitude,Country
5,.ch,POSITIVE,14,46.798562,8.231974,Suisse
21,ARAU,POSITIVE,1,47.392715,8.044445,Suisse
29,Aadorf,POSITIVE,47,47.491578,8.902953,Suisse
30,Aarau,POSITIVE,1151,47.392715,8.044445,Suisse
31,Aarberg,POSITIVE,4,47.044335,7.275300,Suisse


In [290]:
#outFile = "coordinatesForGivenSentiments10m.csv"
#allSuisse.to_csv(outFile,index=None)

In [320]:
allSuisse = pd.read_csv('swissTwitterSentimentsAndCoordinates10m.csv')
allSuisse.head()

,City,Sentiment,Count,Latitude,Longitude,Country
0,.ch,POSITIVE,14,46.798562,8.231974,Suisse
1,ARAU,POSITIVE,1,47.392715,8.044445,Suisse
2,Aadorf,POSITIVE,47,47.491578,8.902953,Suisse
3,Aarau,POSITIVE,1151,47.392715,8.044445,Suisse
4,Aarberg,POSITIVE,4,47.044335,7.275300,Suisse


In [321]:
allSuisse['City']= allSuisse['City'].map(lambda x: x.lower())
#allSuisse = allSuisse.loc[allSuisse.groupby(['City','Sentiment'])["Count"].sum()]
#allSuisse = allSuisse.groupby(['City','Sentiment'])["Count"].sum()
#allSuisse = allSuisse.dropna() 
allSuisse.head()

,City,Sentiment,Count,Latitude,Longitude,Country
0,.ch,POSITIVE,14,46.798562,8.231974,Suisse
1,arau,POSITIVE,1,47.392715,8.044445,Suisse
2,aadorf,POSITIVE,47,47.491578,8.902953,Suisse
3,aarau,POSITIVE,1151,47.392715,8.044445,Suisse
4,aarberg,POSITIVE,4,47.044335,7.275300,Suisse


In [322]:
allSuisse = allSuisse.groupby(['City','Sentiment','Latitude','Longitude'])["Count"].sum()

allSuisse

City           Sentiment  Latitude   Longitude
.ch            POSITIVE   46.798562  8.231974       14
aadorf         POSITIVE   47.491578  8.902953       47
aarau          POSITIVE   47.392715  8.044445     1164
aarberg        POSITIVE   47.044335  7.275300        4
aarburg        POSITIVE   47.320642  7.899360       84
aargau         POSITIVE   47.412396  8.194832     1231
adelboden      POSITIVE   46.492721  7.558762       60
adligenswil    POSITIVE   47.070535  8.368244       10
adliswil       POSITIVE   47.311762  8.524910       33
aegerten       NEGATIVE   47.120241  7.288497        2
aeschi         NEGATIVE   47.180420  7.661874        1
aeugst         POSITIVE   47.533808  7.720602       29
affoltern      POSITIVE   47.418827  8.506867       42
aigle          POSITIVE   46.317901  6.968893     1413
arau           POSITIVE   47.392715  8.044445        1
arisdorf       POSITIVE   47.512381  7.765508        6
arlesheim      POSITIVE   47.493113  7.620205       18
arolla         POS

In [323]:
allSuisse = allSuisse.reset_index()
allSuisse

,City,Sentiment,Latitude,Longitude,Count
0,.ch,POSITIVE,46.798562,8.231974,14
1,aadorf,POSITIVE,47.491578,8.902953,47
2,aarau,POSITIVE,47.392715,8.044445,1164
3,aarberg,POSITIVE,47.044335,7.275300,4
4,aarburg,POSITIVE,47.320642,7.899360,84
5,aargau,POSITIVE,47.412396,8.194832,1231
6,adelboden,POSITIVE,46.492721,7.558762,60
7,adligenswil,POSITIVE,47.070535,8.368244,10
8,adliswil,POSITIVE,47.311762,8.524910,33
9,aegerten,NEGATIVE,47.120241,7.288497,2


In [324]:
def getColour (row):
   if row['Sentiment'] == 'NEUTRAL' :
      return 'blue'
   if row['Sentiment'] == 'POSITIVE' :
      return 'green'
   if row['Sentiment'] == 'NEGATIVE':
      return 'red'
   return 'black'

In [325]:
allSuisse['Colour'] = allSuisse.apply (lambda row: getColour (row),axis=1)
allSuisse.head()

,City,Sentiment,Latitude,Longitude,Count,Colour
0,.ch,POSITIVE,46.798562,8.231974,14,green
1,aadorf,POSITIVE,47.491578,8.902953,47,green
2,aarau,POSITIVE,47.392715,8.044445,1164,green
3,aarberg,POSITIVE,47.044335,7.275300,4,green
4,aarburg,POSITIVE,47.320642,7.899360,84,green


In [340]:
cantons_geo = r'ch-cantons.topojson.json'
canton_map = folium.Map(location=[46.8, 8.33],tiles='OpenStreetMap', zoom_start=8)
canton_map.choropleth(geo_path = cantons_geo, topojson='objects.cantons', fill_color='#3186cc')
canton_map

In [275]:
#mark = simple_marker([x, y], popup='Add <b>popup</b> text here.',marker_color='green',marker_icon='ok-sign')
folium.Marker([x, y],icon=folium.Icon(color='green', icon='ok-sign')).add_to(canton_map)
#canton_map.add_children(mark)
canton_map

NameError: name 'x' is not defined

In [327]:
# Get the location of each city and plot it

for index, row in allSuisse.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row["City"],icon=folium.Icon(color=row["Colour"], icon='ok-sign')).add_to(canton_map)

#canton_map.save( 'mymap.html')


In [328]:
canton_map

In [ ]:
# kanton_map.plot(x,y,marker='o',color='Red',markersize=int(math.sqrt(count))*scale)

In [329]:
canton_map.save( 'mapForTwitterGivenSentiments10months.html')

In [334]:
for index, row in allSuisse.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], radius=row["Count"],popup=row["City"],color=row["Colour"],fill_color=row["Colour"]).add_to(canton_map)


In [335]:
canton_map

In [336]:
canton_map.save( 'mapForTwitterGivenSentiments10monthsRadius.html')

In [337]:
allSuisse['LogCount'] = allSuisse['Count'].map(lambda x: math.log( x ))
allSuisse.head()

,City,Sentiment,Latitude,Longitude,Count,Colour,LogCount
0,.ch,POSITIVE,46.798562,8.231974,14,green,2.639057
1,aadorf,POSITIVE,47.491578,8.902953,47,green,3.850148
2,aarau,POSITIVE,47.392715,8.044445,1164,green,7.059618
3,aarberg,POSITIVE,47.044335,7.275300,4,green,1.386294
4,aarburg,POSITIVE,47.320642,7.899360,84,green,4.430817


In [341]:
for index, row in allSuisse.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], radius=row["LogCount"],popup=row["City"],color=row["Colour"],fill_color=row["Colour"]).add_to(canton_map)


In [342]:
canton_map